# Introducción a Pandas

**Autor:** Roberto Muñoz <br />
**E-mail:** <rmunoz@metricarts.com> <br />
**Github:** <https://github.com/rpmunoz> <br />

[Pandas](http://pandas.pydata.org/) es un paquete de Python que proporciona estructuras de datos similares a los dataframes de R. Pandas depende de Numpy, la librería que añade un potente tipo matricial a Python. Los principales tipos de datos que pueden representarse con pandas son:

- Datos tabulares con columnas de tipo heterogéneo con etiquetas en columnas y filas.
- Series temporales.

Pandas proporciona herramientas que permiten:

- leer y escribir datos en diferentes formatos: CSV, Microsoft Excel, bases SQL y formato HDF5
- seleccionar y filtrar de manera sencilla tablas de datos en función de posición, valor o etiquetas
- fusionar y unir datos
- transformar datos aplicando funciones tanto en global como por ventanas
- manipulación de series temporales
- hacer gráficas

En pandas existen tres tipos básicos de objetos todos ellos basados a su vez en Numpy:

- Series (listas, 1D),
- DataFrame (tablas, 2D) y
- Panels (tablas 3D).

In [1]:
import numpy as np
import pandas as pd
pd.__version__

'0.20.3'

## 1.  Series

A Pandas Series is a one-dimensional array of indexed data. It can be created from a list or array as follows:

In [2]:
data = pd.Series([0.25, 0.5, 0.75, 1.0])
data

0    0.25
1    0.50
2    0.75
3    1.00
dtype: float64

As we see in the output, the Series wraps both a sequence of values and a sequence of indices, which we can access with the values and index attributes. The values are simply a familiar NumPy array:

In [3]:
data.values

array([ 0.25,  0.5 ,  0.75,  1.  ])

The index is an array-like object of type pd.Index, which we'll discuss in more detail momentarily.

In [4]:
data.index

RangeIndex(start=0, stop=4, step=1)

Like with a NumPy array, data can be accessed by the associated index via the familiar Python square-bracket notation:

In [5]:
data[1]

0.5

### Series as generalized NumPy array

From what we've seen so far, it may look like the Series object is basically interchangeable with a one-dimensional NumPy array. The essential difference is the presence of the index: while the Numpy Array has an implicitly defined integer index used to access the values, the Pandas Series has an explicitly defined index associated with the values.

In [6]:
data = pd.Series([0.25, 0.5, 0.75, 1.0],
                 index=['a', 'b', 'c', 'd'])
data

a    0.25
b    0.50
c    0.75
d    1.00
dtype: float64

And the item access works as expected:

In [7]:
data['b']

0.5

### Series as specialized dictionary

In this way, you can think of a Pandas Series a bit like a specialization of a Python dictionary. A dictionary is a structure that maps arbitrary keys to a set of arbitrary values, and a Series is a structure which maps typed keys to a set of typed values. This typing is important: just as the type-specific compiled code behind a NumPy array makes it more efficient than a Python list for certain operations, the type information of a Pandas Series makes it much more efficient than Python dictionaries for certain operations.

In [8]:
population_dict = {'Arica y Parinacota': 243149,
                   'Antofagasta': 631875,
                   'Metropolitana de Santiago': 7399042,
                   'Valparaiso': 1842880,
                   'Bíobío': 2127902,
                   'Magallanes y Antártica Chilena': 165547}
population = pd.Series(population_dict)
population

Antofagasta                        631875
Arica y Parinacota                 243149
Bíobío                            2127902
Magallanes y Antártica Chilena     165547
Metropolitana de Santiago         7399042
Valparaiso                        1842880
dtype: int64

You can notice the indexes were sorted lexicographically. That's the default behaviour in Pandas

In [9]:
population['Arica y Parinacota']

243149

Unlike a dictionary, though, the Series also supports array-style operations such as slicing:

In [10]:
population['Metropolitana':'Valparaíso']

Metropolitana de Santiago    7399042
Valparaiso                   1842880
dtype: int64

## 2. DataFrame

The next fundamental structure in Pandas is the DataFrame. Like the Series object discussed in the previous section, the DataFrame can be thought of either as a generalization of a NumPy array, or as a specialization of a Python dictionary. We'll now take a look at each of these perspectives.

### DataFrame as a generalized NumPy array

If a Series is an analog of a one-dimensional array with flexible indices, a DataFrame is an analog of a two-dimensional array with both flexible row indices and flexible column names.

In [11]:
# Area in km^2
area_dict = {'Arica y Parinacota': 16873.3,
             'Antofagasta': 126049.1,
             'Metropolitana de Santiago': 15403.2,
             'Valparaiso': 16396.1,
             'Bíobío': 37068.7,
             'Magallanes y Antártica Chilena': 1382291.1}
area = pd.Series(area_dict)
area

Antofagasta                        126049.1
Arica y Parinacota                  16873.3
Bíobío                              37068.7
Magallanes y Antártica Chilena    1382291.1
Metropolitana de Santiago           15403.2
Valparaiso                          16396.1
dtype: float64

Now that we have this along with the population Series from before, we can use a dictionary to construct a single two-dimensional object containing this information:

In [12]:
regions = pd.DataFrame({'population': population,
                       'area': area})
regions

,area,population
Antofagasta,126049.1,631875
Arica y Parinacota,16873.3,243149
Bíobío,37068.7,2127902
Magallanes y Antártica Chilena,1382291.1,165547
Metropolitana de Santiago,15403.2,7399042
Valparaiso,16396.1,1842880


In [13]:
regions.index

Index(['Antofagasta', 'Arica y Parinacota', 'Bíobío',
       'Magallanes y Antártica Chilena', 'Metropolitana de Santiago',
       'Valparaiso'],
      dtype='object')

In [14]:
regions.columns

Index(['area', 'population'], dtype='object')

### DataFrame as specialized dictionary

Similarly, we can also think of a DataFrame as a specialization of a dictionary. Where a dictionary maps a key to a value, a DataFrame maps a column name to a Series of column data. For example, asking for the 'area' attribute returns the Series object containing the areas we saw earlier:

In [15]:
regions['area']

Antofagasta                        126049.1
Arica y Parinacota                  16873.3
Bíobío                              37068.7
Magallanes y Antártica Chilena    1382291.1
Metropolitana de Santiago           15403.2
Valparaiso                          16396.1
Name: area, dtype: float64

### Constructing DataFrame objects
A Pandas DataFrame can be constructed in a variety of ways. Here we'll give several examples.

### From a single Series object¶
A DataFrame is a collection of Series objects, and a single-column DataFrame can be constructed from a single Series:

In [16]:
pd.DataFrame(population, columns=['poblacion'])

,poblacion
Antofagasta,631875
Arica y Parinacota,243149
Bíobío,2127902
Magallanes y Antártica Chilena,165547
Metropolitana de Santiago,7399042
Valparaiso,1842880


### From a dictionary of Series objects
As we saw before, a DataFrame can be constructed from a dictionary of Series objects as well:

In [17]:
pd.DataFrame({'poblacion': population,
              'area': area}, columns=['poblacion', 'area'])

,poblacion,area
Antofagasta,631875,126049.1
Arica y Parinacota,243149,16873.3
Bíobío,2127902,37068.7
Magallanes y Antártica Chilena,165547,1382291.1
Metropolitana de Santiago,7399042,15403.2
Valparaiso,1842880,16396.1


## 3. Reading a CSV file and doing common Pandas operations

In [18]:
regiones_file='data/chile_regiones.csv'
provincias_file='data/chile_provincias.csv'
comunas_file='data/chile_comunas.csv'

regiones=pd.read_csv(regiones_file, header=0, sep=',')

provincias=pd.read_csv(provincias_file, header=0, sep=',')
comunas=pd.read_csv(comunas_file, header=0, sep=',')

In [19]:
print('regiones table: ', regiones.columns.values.tolist())
print('provincias table: ', provincias.columns.values.tolist())
print('comunas table: ', comunas.columns.values.tolist())

regiones table:  ['RegionID', 'RegionNombre', 'RegionOrdinal']
provincias table:  ['ProvinciaID', 'ProvinciaNombre', 'RegionID']
comunas table:  ['ComunaID', 'ComunaNombre', 'ProvinciaID']


In [20]:
regiones.head()

,RegionID,RegionNombre,RegionOrdinal
0,1,'Arica y Parinacota','XV'
1,2,'Tarapacá','I'
2,3,'Antofagasta','II'
3,4,'Atacama','III'
4,5,'Coquimbo','IV'


In [21]:
provincias.head()

,ProvinciaID,ProvinciaNombre,RegionID
0,1,'Arica',1
1,2,'Parinacota',1
2,3,'Iquique',2
3,4,'El Tamarugal',2
4,5,'Antofagasta',3


In [22]:
comunas.describe()

,ComunaID,ProvinciaID
count,345.000000,345.000000
mean,173.000000,29.368116
std,99.737155,11.853791
min,1.000000,1.000000
25%,87.000000,22.000000
50%,173.000000,31.000000
75%,259.000000,38.000000
max,345.000000,53.000000


In [23]:
regiones_provincias=pd.merge(regiones, provincias, how='outer')
regiones_provincias.head()

,RegionID,RegionNombre,RegionOrdinal,ProvinciaID,ProvinciaNombre
0,1,'Arica y Parinacota','XV',1,'Arica'
1,1,'Arica y Parinacota','XV',2,'Parinacota'
2,2,'Tarapacá','I',3,'Iquique'
3,2,'Tarapacá','I',4,'El Tamarugal'
4,3,'Antofagasta','II',5,'Antofagasta'


In [24]:
provincias_comunas=pd.merge(provincias, comunas, how='outer')
provincias_comunas.head()

,ProvinciaID,ProvinciaNombre,RegionID,ComunaID,ComunaNombre
0,1,'Arica',1,1,'Arica'
1,1,'Arica',1,2,'Camarones'
2,2,'Parinacota',1,3,'General Lagos'
3,2,'Parinacota',1,4,'Putre'
4,3,'Iquique',2,5,'Alto Hospicio'


In [25]:
regiones_provincias_comunas=pd.merge(regiones_provincias, comunas, how='outer')
regiones_provincias_comunas.index.name='ID'
regiones_provincias_comunas.head()

,RegionID,RegionNombre,RegionOrdinal,ProvinciaID,ProvinciaNombre,ComunaID,ComunaNombre
ID,,,,,,,
0,1,'Arica y Parinacota','XV',1,'Arica',1,'Arica'
1,1,'Arica y Parinacota','XV',1,'Arica',2,'Camarones'
2,1,'Arica y Parinacota','XV',2,'Parinacota',3,'General Lagos'
3,1,'Arica y Parinacota','XV',2,'Parinacota',4,'Putre'
4,2,'Tarapacá','I',3,'Iquique',5,'Alto Hospicio'


In [26]:
regiones_provincias_comunas.to_csv('data/chile_demographic_merge.csv', index=False)

In [27]:
regiones_provincias_comunas.loc[3]

RegionID                              1
RegionNombre       'Arica y Parinacota'
RegionOrdinal                      'XV'
ProvinciaID                           2
ProvinciaNombre            'Parinacota'
ComunaID                              4
ComunaNombre                    'Putre'
Name: 3, dtype: object

## 4. Loading ful dataset

In [28]:
data_file='data/chile_demographic.csv'
data=pd.read_csv(data_file, header=0, sep=',')
data.head()

,RegionID,Region,Provincia,Comuna,Superficie,Poblacion,Densidad,IDH_2005
0,1,Arica y Parinacota,Arica,Arica,4799.4,210936,38.4,0.736
1,1,Arica y Parinacota,Arica,Camarones,3927.0,679,0.3,0.751
2,1,Arica y Parinacota,Parinacota,General Lagos,2244.4,739,0.5,0.670
3,1,Arica y Parinacota,Parinacota,Putre,5902.5,1462,0.2,0.707
4,1,Arica y Parinacota,Iquique,Alto Hospicio,572.9,94455,87.6,NaN


In [29]:
# Podemos ordenar el dataframe usando el campo Poblacion

data_sort=data.sort_values('Poblacion')
data_sort.head()

,RegionID,Region,Provincia,Comuna,Superficie,Poblacion,Densidad,IDH_2005
335,15,Magallanes y de la Antártica Chilena,Antártica Chilena,Antártica,1250257.6,127,0.0,NaN
15,3,Antofagasta,El Loa,Ollagüe,2964.0,332,0.1,0.679
342,15,Magallanes y de la Antártica Chilena,Magallanes,Río Verde,17248.0,363,0.0,0.784
334,14,Aisén del General Carlos Ibáñez del Campo,Capitán Prat,Tortel,19710.6,531,0.0,0.655
337,15,Magallanes y de la Antártica Chilena,Magallanes,Laguna Blanca,3695.6,631,0.0,0.785


In [30]:
# Podemos ordenarlo de mayor a menor

data_sort=data.sort_values('Poblacion', ascending=False)
data_sort.head()

,RegionID,Region,Provincia,Comuna,Superficie,Poblacion,Densidad,IDH_2005
107,7,Metropolitana de Santiago,Santiago,Maipú,135.5,805000,3876.2,0.902
119,7,Metropolitana de Santiago,Cordillera,Puente Alto,88.0,757721,6664.8,0.773
97,7,Metropolitana de Santiago,Santiago,La Florida,70.2,397497,5209.0,0.804
11,3,Antofagasta,Antofagasta,Antofagasta,30718.1,348669,9.7,0.734
80,6,Valparaíso,Valparaíso,Viña del Mar,121.6,311399,2560.8,0.766


In [31]:
(data.groupby(data['Region'])['Poblacion','Superficie'].sum())

,Poblacion,Superficie
Region,,
Aisén del General Carlos Ibáñez del Campo,106893,107449.40
Antofagasta,551627,126048.80
Arica y Parinacota,308271,17446.20
Atacama,292054,74806.30
Biobío,2025995,37068.70
Coquimbo,714856,40967.80
La Araucanía,933537,31842.30
Libertador General Bernardo O'Higgins,903248,16583.30
Los Lagos,835829,48583.30


In [32]:
x=data.groupby(data['Region'])
x['Poblacion','Superficie'].sum()

,Poblacion,Superficie
Region,,
Aisén del General Carlos Ibáñez del Campo,106893,107449.40
Antofagasta,551627,126048.80
Arica y Parinacota,308271,17446.20
Atacama,292054,74806.30
Biobío,2025995,37068.70
Coquimbo,714856,40967.80
La Araucanía,933537,31842.30
Libertador General Bernardo O'Higgins,903248,16583.30
Los Lagos,835829,48583.30


In [33]:
(data.groupby(data['Region'])['Poblacion','Superficie'].sum()) \
    .sort_values(['Poblacion'])

,Poblacion,Superficie
Region,,
Aisén del General Carlos Ibáñez del Campo,106893,107449.40
Magallanes y de la Antártica Chilena,158828,1392783.70
Tarapacá,205566,41632.60
Atacama,292054,74806.30
Arica y Parinacota,308271,17446.20
Los Ríos,380618,18577.60
Antofagasta,551627,126048.80
Coquimbo,714856,40967.80
Los Lagos,835829,48583.30
